In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
#%matplotlib inline
from bs4 import BeautifulSoup
import html5lib
import requests
import datetime
import time

In [2]:
path = 'http://ergast.com/api/f1'
season = '2016'
race = '3'
URL = path + '/' + season + '/' + race + '/' + 'laps?limit=1000'

#Download
# results = requests.get(URL)
# soup = BeautifulSoup(results.content, 'lxml')
# print("First 1000 records downloaded")
# #Part 2
# URL = path + '/' + season + '/' + race + '/' + 'laps?limit=1000&offset=1000'
# results2 = requests.get(URL)
# soup2 = BeautifulSoup(results.content, 'lxml')

# with open('china_race_dl.xml', 'wb') as f:
#     f.write(results.content)
#     f.write(results2.content)


#open saved copy
with open('china_race_dl.xml', 'rb') as f:
    results = f.read()
soup = BeautifulSoup(results, 'lxml')


In [3]:
#pull in driverids to names
driver_info = pd.DataFrame.from_csv('driver_info.csv')
driverid_dict = {driver_info.ix[i]['driverid']:driver_info.ix[i]['GivenName'] + " " + driver_info.ix[i]['FamilyName'] for i in driver_info.index}
driverid_dict['sainz'] = driverid_dict['sainz'] + ' Jr.'
colors = {driver_info.ix[i]['GivenName'] + " " + driver_info.ix[i]['FamilyName']: driver_info.ix[i]['color'] for i in driver_info.index}
colors['Carlos Sainz Jr.'] = colors['Carlos Sainz']


In [4]:
driver_abrev = {driver_info.ix[i]['GivenName'] + " " + driver_info.ix[i]['FamilyName']:driver_info.ix[i]['code'] for i in driver_info.index}
driver_abrev['Carlos Sainz Jr.'] = 'SAI'
driver_abrev

{'Carlos Sainz': 'SAI',
 'Carlos Sainz Jr.': 'SAI',
 'Daniel Ricciardo': 'RIC',
 'Daniil Kvyat': 'KVY',
 'Esteban Gutiérrez': 'GUT',
 'Felipe Massa': 'MAS',
 'Felipe Nasr': 'NAS',
 'Fernando Alonso': 'ALO',
 'Jenson Button': 'BUT',
 'Jolyon Palmer': 'PAL',
 'Kevin Magnussen': 'MAG',
 'Kimi Räikkönen': 'RAI',
 'Lewis Hamilton': 'HAM',
 'Marcus Ericsson': 'ERI',
 'Max Verstappen': 'VES',
 'Nico Hülkenberg': 'HUL',
 'Nico Rosberg': 'ROS',
 'Pascal Wehrlein': 'WEH',
 'Rio Haryanto': 'HAR',
 'Romain Grosjean': 'GRO',
 'Sebastian Vettel': 'VET',
 'Sergio Pérez': 'PER',
 'Stoffel Vandoorne': 'VAN',
 'Valtteri Bottas': 'BOT'}

In [5]:
headers = ('driverid', 'lap', 'time')
num_rows = len(soup.find_all('timing'))
race_dict = {}
for header in headers:
    race_dict[header] = [race[header] for race in soup.find_all('timing')]
df = pd.DataFrame(race_dict)
df['minutes'] = [x.split(":")[0] for x in df['time']]
df['seconds'] = [x.split(":")[1] for x in df['time']]
df['lap_sec'] = df['minutes'].astype('int64') * 60 + df['seconds'].astype('float32')
df['lap'] = df['lap'].astype('int64')
df['driver'] = [driverid_dict[driver] for driver in df['driverid']]
df

,driverid,lap,time,minutes,seconds,lap_sec,driver
0,ricciardo,1,1:43.315,1,43.315,103.314999,Daniel Ricciardo
1,rosberg,1,1:44.084,1,44.084,104.084000,Nico Rosberg
2,kvyat,1,1:45.491,1,45.491,105.491001,Daniil Kvyat
3,perez,1,1:46.374,1,46.374,106.374001,Sergio Pérez
4,hulkenberg,1,1:46.981,1,46.981,106.980999,Nico Hülkenberg
5,sainz,1,1:47.539,1,47.539,107.539001,Carlos Sainz Jr.
6,massa,1,1:48.086,1,48.086,108.085999,Felipe Massa
7,vettel,1,1:48.651,1,48.651,108.651001,Sebastian Vettel
8,button,1,1:49.024,1,49.024,109.023998,Jenson Button
9,bottas,1,1:49.323,1,49.323,109.323002,Valtteri Bottas


In [6]:
pivot = df.pivot(index='driver', columns='lap', values='lap_sec')
pivot

lap,1,2,3,4,5,6,7,8,9,10,...,47,48,49,50,51,52,53,54,55,56
driver,,,,,,,,,,,,,,,,,,,,,
Carlos Sainz Jr.,107.539001,103.848000,103.862000,120.821000,153.591999,133.195000,145.070000,148.598000,104.330002,105.603001,...,102.921001,103.931999,102.973000,102.807999,102.584999,102.175999,102.169998,103.313999,104.414001,103.243999
Daniel Ricciardo,103.314999,102.308998,116.348999,139.944000,135.444000,131.770000,146.257999,143.545000,104.396999,104.923000,...,101.203999,101.174000,101.333000,101.629002,101.570000,101.764000,101.187000,101.103001,101.014999,101.817001
Daniil Kvyat,105.491001,103.018002,103.507999,115.312000,151.966000,137.191999,145.898001,151.068001,102.758999,102.978001,...,101.827000,102.179001,102.067001,102.291000,102.032001,102.050999,102.277000,102.848999,102.372002,102.234001
Esteban Gutiérrez,113.193001,105.589001,106.284000,117.862999,135.601000,136.757000,145.761999,152.021999,105.513000,104.987999,...,100.971001,101.165001,101.318001,101.540001,101.895000,101.464001,101.463001,101.778999,102.202000,NaN
Felipe Massa,108.085999,105.507000,104.619999,116.504002,135.552000,142.252001,145.903000,152.738998,103.330002,103.130001,...,103.202999,103.034000,102.905998,102.735001,102.569000,102.786999,102.826000,103.201000,103.694000,104.175999
Felipe Nasr,170.226002,136.614000,113.287998,144.577999,146.598000,113.820000,104.557999,104.629002,105.057999,105.584000,...,103.011002,103.191002,103.820999,103.734001,105.622002,105.570000,105.323002,104.721001,106.444000,NaN
Fernando Alonso,111.011002,105.014000,104.830002,116.784000,136.646999,138.916000,145.974001,152.792999,104.780998,104.233002,...,102.484001,102.935001,102.622002,102.623001,103.077999,102.779999,102.958000,103.248001,103.061001,103.522999
Jenson Button,109.023998,103.980000,103.616001,119.945999,152.333000,132.960000,145.399000,148.667000,104.661999,104.783001,...,100.509998,101.073002,101.224998,101.855000,101.800999,102.811001,103.209000,103.789001,104.277000,104.882999
Jolyon Palmer,112.610001,105.884998,106.105000,119.143002,136.823999,136.737000,146.122000,150.639999,106.318001,107.265999,...,103.325001,103.050999,103.185001,104.247002,103.314999,103.290001,103.587002,103.453999,103.374001,NaN


In [7]:
for i in range(2, df['lap'].max() + 1):
    pivot.ix[:,i] = pivot.ix[:,i] + pivot.ix[:, i-1]
pivot

lap,1,2,3,4,5,6,7,8,9,10,...,47,48,49,50,51,52,53,54,55,56
driver,,,,,,,,,,,,,,,,,,,,,
Carlos Sainz Jr.,107.539001,211.387001,315.249001,436.070001,589.662000,722.856999,867.926999,1016.524999,1120.855000,1226.458001,...,5090.402013,5194.334012,5297.307012,5400.115010,5502.700010,5604.876008,5707.046006,5810.360006,5914.774007,6018.018007
Daniel Ricciardo,103.314999,205.623997,321.972996,461.916996,597.360996,729.130997,875.388996,1018.933996,1123.330996,1228.253996,...,5073.986990,5175.160989,5276.493990,5378.122991,5479.692991,5581.456991,5682.643991,5783.746992,5884.761991,5986.578993
Daniil Kvyat,105.491001,208.509003,312.017002,427.329002,579.295002,716.487001,862.385002,1013.453003,1116.212002,1219.190002,...,5059.474993,5161.653994,5263.720995,5366.011996,5468.043997,5570.094996,5672.371996,5775.220995,5877.592997,5979.826998
Esteban Gutiérrez,113.193001,218.782001,325.066002,442.929001,578.530001,715.287001,861.049000,1013.070999,1118.584000,1223.571999,...,5125.710998,5226.875999,5328.194000,5429.734001,5531.629001,5633.093002,5734.556003,5836.335003,5938.537002,NaN
Felipe Massa,108.085999,213.592999,318.212997,434.716999,570.268999,712.521000,858.424000,1011.162998,1114.493000,1217.623001,...,5081.474010,5184.508010,5287.414009,5390.149009,5492.718010,5595.505008,5698.331008,5801.532009,5905.226009,6009.402008
Felipe Nasr,170.226002,306.840002,420.128000,564.705999,711.303999,825.123999,929.681997,1034.310999,1139.368998,1244.952997,...,5151.236003,5254.427005,5358.248004,5461.982005,5567.604007,5673.174006,5778.497008,5883.218009,5989.662009,NaN
Fernando Alonso,111.011002,216.025002,320.855003,437.639004,574.286003,713.202003,859.176004,1011.969004,1116.750002,1220.983004,...,5104.315999,5207.251000,5309.873002,5412.496003,5515.574002,5618.354001,5721.312001,5824.560002,5927.621003,6031.144002
Jenson Button,109.023998,213.003998,316.619999,436.565998,588.898998,721.858998,867.257998,1015.924998,1120.586997,1225.369998,...,5110.957992,5212.030994,5313.255993,5415.110992,5516.911991,5619.722992,5722.931991,5826.720993,5930.997993,6035.880993
Jolyon Palmer,112.610001,218.494999,324.599998,443.743000,580.566999,717.303999,863.425999,1014.065998,1120.383999,1227.649998,...,5169.865986,5272.916985,5376.101986,5480.348988,5583.663986,5686.953987,5790.540989,5893.994988,5997.368988,NaN


In [8]:
unpivot = pivot.stack()
new_df = unpivot.reset_index()
new_df.columns = ['driverid', 'lap', 'cume_sec']
new_df.sort_values('cume_sec', ascending=True, inplace=True)
new_df = new_df.reset_index()
new_df.drop('index', axis=1, inplace=True)
new_df

,driverid,lap,cume_sec
0,Daniel Ricciardo,1,103.314999
1,Nico Rosberg,1,104.084000
2,Daniil Kvyat,1,105.491001
3,Sergio Pérez,1,106.374001
4,Nico Hülkenberg,1,106.980999
5,Carlos Sainz Jr.,1,107.539001
6,Felipe Massa,1,108.085999
7,Sebastian Vettel,1,108.651001
8,Jenson Button,1,109.023998
9,Valtteri Bottas,1,109.323002


In [9]:
from f1 import Team
from f1 import Race

In [10]:
#Set up a Race object, score it, return a dataframe of all points
ThisRace = Race('China')
df_score = ThisRace.score()
df_score

C:\users\jason\documents\gitrepos\fantasy-formula-one\f1.py:223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  fantasy_points['Team_Pts'][fantasy_points['Finish'].isin(team_points)] = 1
C:\users\jason\documents\gitrepos\fantasy-formula-one\f1.py:227: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  fantasy_points['Movement_Pts'][fantasy_points['Movement_Pts'] < 0] = 0
C:\users\jason\documents\gitrepos\fantasy-formula-one\f1.py:228: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  fantasy_poi

,Team,Qualifying,Grid,Eff_Grid,Finish,Qual_Pts,Fin_Pts,Team_Pts,Movement_Pts,Completion,Fst_Lap,Total_Race_Pts
Sebastian Vettel,Ferrari,4,4,4,2,2,9,1,2,3,0,17
Kimi Räikkönen,Ferrari,3,3,3,5,2,6,0,0,3,0,11
Sergio Pérez,Force India-Mercedes,7,7,7,11,2,0,1,0,3,0,6
Nico Hülkenberg,Force India-Mercedes,10,13,13,15,2,0,0,0,3,2,7
Esteban Gutiérrez,Haas-Ferrari,18,18,18,14,0,0,1,4,3,0,8
Romain Grosjean,Haas-Ferrari,14,14,14,19,1,0,0,0,3,0,4
Pascal Wehrlein,MRT-Mercedes,21,21,21,18,0,0,1,3,3,0,7
Rio Haryanto,MRT-Mercedes,20,20,20,21,0,0,0,0,3,0,3
Fernando Alonso,McLaren-Honda,12,11,11,12,1,0,1,0,3,0,5
Jenson Button,McLaren-Honda,13,12,12,13,1,0,0,0,3,0,4


In [11]:
ThisRace.lap_length

56

In [12]:
#Set up current dataframe
current = pd.DataFrame(columns=['position', 'lap'])
current['position'] = df_score['Grid'].copy()
current['lap'] = 0
if 'DNS' in current['position']:
    current['position'][current['position'] == 'DNS'] = np.NaN
if 'PL' in current['position']:
    current['position'][current['position'] == 'PL'] = np.NaN
current.sort_values('position', inplace=True)
current

,position,lap
Nico Rosberg,1,0
Daniel Ricciardo,2,0
Kimi Räikkönen,3,0
Sebastian Vettel,4,0
Valtteri Bottas,5,0
Daniil Kvyat,6,0
Sergio Pérez,7,0
Carlos Sainz Jr.,8,0
Max Verstappen,9,0
Felipe Massa,10,0


In [13]:
#Set all laps completed to zero
ThisRace.laps_completed = {driver:current.ix[driver]['lap'] for driver in current.index}
ThisRace.laps_completed

{'Carlos Sainz Jr.': 0,
 'Daniel Ricciardo': 0,
 'Daniil Kvyat': 0,
 'Esteban Gutiérrez': 0,
 'Felipe Massa': 0,
 'Felipe Nasr': 0,
 'Fernando Alonso': 0,
 'Jenson Button': 0,
 'Jolyon Palmer': 0,
 'Kevin Magnussen': 0,
 'Kimi Räikkönen': 0,
 'Lewis Hamilton': 0,
 'Marcus Ericsson': 0,
 'Max Verstappen': 0,
 'Nico Hülkenberg': 0,
 'Nico Rosberg': 0,
 'Pascal Wehrlein': 0,
 'Rio Haryanto': 0,
 'Romain Grosjean': 0,
 'Sebastian Vettel': 0,
 'Sergio Pérez': 0,
 'Valtteri Bottas': 0}

In [14]:
import matplotlib.animation as animation
fastest_lap_time = 1000
first = False

def update_driver_scores2(step, rects, ax):
    global fastest_lap_time
    global ThisRace
    global current
    global new_df
    global first
    if first == False:
        first = True
    else:
        driver, lap, cume_sec = new_df.ix[step]
        if lap == 1:
            if cume_sec != np.NaN:
                cur_lap_time = cume_sec
        elif new_df.ix[i, 'cume_sec'] != np.NaN:
            cur_lap_time = cume_sec - float(new_df['cume_sec'][(new_df['driverid'] == driver) & (new_df['lap'] == lap - 1)])

        if cur_lap_time < fastest_lap_time: #check for fastest lap
            ThisRace.fastest_lap = driver
            fastest_lap_time = cur_lap_time
        current.ix[driver, 'lap'] += 1 #increment lap
        current.sort_values('lap', kind='mergesort', inplace=True, ascending=False) #sort by lap, stable sort
        current['position'] = np.arange(1,len(current) + 1) #recalc positions
        ThisRace.laps_completed[driver] = lap #update laps completed for driver
        ThisRace.drivers_position = {driver:current.ix[driver, 'position'] for driver in current.index} #update driver positions
        ThisRace.score().sort_values('Finish', inplace=True, ascending=True)   
        for rect, h, driver in zip(rects, 
                                   [ThisRace.fantasy_points[driver] for driver in current.index], 
                                   [driver for driver in current.index]):
            rect.set_height(h)
            rect.set_color(colors[driver])
        ax.xaxis.set_ticklabels([driver_abrev[driver] for driver in current.index])
        #ax.xaxis.set_ticklabels(current.index)
start_time = datetime.datetime.now()

#whole race
end_time = start_time + datetime.timedelta(seconds=float(new_df[-1:]['cume_sec']))
ThisRace.drivers_position = ThisRace.grid_position
ThisRace.score()

fig1 = plt.figure()
#fig, ax = plt.subplots()
ax = fig1.add_subplot(1,1,1)
x_pos = current['position']
points = [ThisRace.fantasy_points[driver] for driver in current.index]
rects = ax.bar(x_pos, points, align='center', alpha=0.7)
plt.xticks(x_pos, current.index, rotation=90)
plt.xlim([0, len(points)+1])
plt.title('Drivers Fantasy Points')
plt.ylabel('Points')
plt.ylim([0, 25])
ani = animation.FuncAnimation(fig1, update_driver_scores2, 50, fargs=([rects, ax]))
ani.save('China.mp4', fps=1)

C:\users\jason\documents\gitrepos\fantasy-formula-one\f1.py:223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  fantasy_points['Team_Pts'][fantasy_points['Finish'].isin(team_points)] = 1
C:\users\jason\documents\gitrepos\fantasy-formula-one\f1.py:227: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  fantasy_points['Movement_Pts'][fantasy_points['Movement_Pts'] < 0] = 0
C:\users\jason\documents\gitrepos\fantasy-formula-one\f1.py:228: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  fantasy_poi

In [15]:
current

,position,lap
Nico Rosberg,1,3
Daniil Kvyat,2,3
Sergio Pérez,3,3
Nico Hülkenberg,4,3
Carlos Sainz Jr.,5,3
Sebastian Vettel,6,3
Daniel Ricciardo,7,2
Jenson Button,8,2
Felipe Massa,9,2
Valtteri Bottas,10,2


In [ ]:
ThisRace.score()

In [ ]:
fastest_lap_time = 1000
start_time = datetime.datetime.now()
#whole race

end_time = start_time + datetime.timedelta(seconds=float(new_df[-1:]['cume_sec']))
#2 min test
#end_time = start_time + datetime.timedelta(seconds=120)
print(start_time.isoformat())
print(end_time.isoformat())
i = 0
#Initialize graph
drivers = [driver for driver in current.index]
x_pos = np.arange(len(current))
points = [ThisRace.fantasy_points[driver] for driver in drivers]
plt.ion()
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
plt.bar(x_pos, points, align='center', alpha=0.7)
plt.xticks(x_pos, drivers, rotation=90)
plt.xlim([-1, len(points)])
plt.title('Drivers Fantasy Points')
plt.xlabel('Driver')
plt.ylabel('Points')
plt.ylim([0, 25])
plt.draw()
#for i in range(300):
while datetime.datetime.now() < end_time:
    if datetime.datetime.now() >= start_time + datetime.timedelta(seconds=new_df.ix[i, 'cume_sec']) or i < 900:
    #if i <= 300:
        if new_df.ix[i, 'cume_sec'] != np.NaN:
            #print(new_df.ix[i, 'driverid'], 'crossed the line! ', new_df.ix[i, 'cume_sec'])
            cur_driver = new_df.ix[i, 'driverid']
            cur_lap = new_df.ix[i, 'lap']
            current.ix[cur_driver, 'lap'] += 1 #increment lap
            current.sort_values('lap', kind='mergesort', inplace=True, ascending=False) #sort by lap, stable
            current['position'] = np.arange(1,len(current) + 1) #recalc positions
            ThisRace.laps_completed[cur_driver] += 1
            ThisRace.drivers_position = {driver:current.ix[driver, 'position'] for driver in current.index}
            #print(type(cur_lap), type(new_df.ix[i, 'cume_sec']), type(np.NaN))
            if cur_lap == 1:
                if new_df.ix[i, 'cume_sec'] != np.NaN:
                    cur_lap_time = new_df.ix[i, 'cume_sec']
                    print('Current lap time:', cur_lap_time, cur_driver)
            elif new_df.ix[i, 'cume_sec'] != np.NaN:
                #print(new_df.ix[i, 'cume_sec'])
                #print(float(new_df['cume_sec'][(new_df['driverid'] == cur_driver) & (new_df['lap'] == cur_lap - 1)]))
                cur_lap_time = new_df.ix[i, 'cume_sec'] - float(new_df['cume_sec'][(new_df['driverid'] == cur_driver) & (new_df['lap'] == cur_lap - 1)])
                print('Current lap time:', cur_lap_time, cur_driver)

            #print(new_df['cume_sec'][(new_df['driverid'] == cur_driver) & (new_df['lap'] == 5)])
            if cur_lap_time < fastest_lap_time:
                ThisRace.fastest_lap = cur_driver
                fastest_lap_time = cur_lap_time
                print(cur_driver, 'got the fastest lap!')
            df_score = ThisRace.score()
            df_score.sort_values('Finish', inplace=True, ascending=True)
            #update graph
            drivers = [driver for driver in current.index]
            x_pos = np.arange(len(current))
            points = [ThisRace.fantasy_points[driver] for driver in drivers]
            ax.clear()
            plt.bar(x_pos, points, align='center', alpha=0.7)
            plt.xticks(x_pos, drivers, rotation=90)
            plt.pause(.05)
        i += 1
df_score['Total_Race_Pts'].plot(kind='bar')
current
        

In [ ]:
import matplotlib.animation as animation

def update_line(num, data, line):
    line.set_data(data[...,:num])
    return line,

fig1 = plt.figure()

data = np.random.rand(2, 25)
l, = plt.plot([], [], 'r-')
plt.xlim(0, 1)
plt.ylim(0, 1)
plt.xlabel('x')
plt.title('test')
line_ani = animation.FuncAnimation(fig1, update_line, 25, fargs=(data, l),
    interval=50, blit=True)
#line_ani.save('lines.mp4')

fig2 = plt.figure()

x = np.arange(-9, 10)
y = np.arange(-9, 10).reshape(-1, 1)
base = np.hypot(x, y)
ims = []
for add in np.arange(15):
    ims.append((plt.pcolor(x, y, base + add, norm=plt.Normalize(0, 30)),))

im_ani = animation.ArtistAnimation(fig2, ims, interval=50, repeat_delay=3000,
    blit=True)
#im_ani.save('im.mp4', metadata={'artist':'Guido'})

plt.show()

In [ ]:
import matplotlib.animation as animation
def update_driver_scores(raceObj, current, driver, lap, lap_time, abs_time):
    #do I need globals so the objects are edited?
    current.ix[driver, 'lap'] += 1 #increment lap
    current.sort_values('lap', kind='mergesort', inplace=True, ascending=False) #sort by lap, stable
    current['position'] = np.arange(1,len(current) + 1) #recalc positions
    raceObj.laps_completed[driver] = lap #update laps completed for driver
    raceObj.drivers_position = {driver:current.ix[driver, 'position'] for driver in current.index} #update driver positions
    if lap_time < fastest_lap_time: #check for fastest lap
        raceObj.fastest_lap = driver
        fastest_lap_time = cur_lap_time
    return raceObj.score().sort_values('Finish', inplace=True, ascending=True)

def update_bar_chart(racePanda, rects):
    for rect, h in zip(rects, racePanda['Total_Race_Pts']):
        rect.set_height(h)

fastest_lap_time = 1000
start_time = datetime.datetime.now()
#whole race
end_time = start_time + datetime.timedelta(seconds=float(new_df[-1:]['cume_sec']))
i = 0
ThisRace.drivers_position = ThisRace.grid_position
ThisRace.score()

fig1 = plt.figure()
ax = fig1.add_subplot(1,1,1)
x_pos = current['position']
points = [ThisRace.fantasy_points[driver] for driver in current.index]
rects = plt.bar(x_pos, points, align='center', alpha=0.7)
plt.xticks(x_pos, current.index, rotation=90)
plt.xlim([-1, len(points)])
plt.title('Drivers Fantasy Points')
plt.xlabel('Driver')
plt.ylabel('Points')
plt.ylim([0, 25])
#ani = animation.FuncAnimation(fig1, update_driver_scores, i, fargs=(ThisRace, ))
plt.show()

In [ ]:
type(rects)

In [ ]:
ThisRace.lap_length